In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import time
import json
import csv

# Function to search for a track on Spotify
def search_track(query, access_token):
    api_url = "https://api.spotify.com/v1/search"
    params = {
        "q": query,
        "type": "track",
        "limit": 1
    }
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    max_retry_attempts = 5
    retry_delay = 5
    retry_attempts = 0

    while retry_attempts < max_retry_attempts:
        try:
            response = requests.get(api_url, params=params, headers=headers)

            if response.status_code == 200:
                data = response.json()
                if data.get('tracks') and data['tracks'].get('items'):
                    track_info = data['tracks']['items'][0]
                    return track_info
                else:
                    print(f"No track found for query: {query}")
                    return None
            elif response.status_code == 429:
                retry_delay *= 2
                print(f"Rate limited. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_attempts += 1
            else:
                print(f"Error for query '{query}': {response.status_code} - {response.text}")
                break

        except Exception as e:
            print(f"Exception for query '{query}': {str(e)}")

    raise RuntimeError(f"Error while fetching track information for query '{query}'")

# Function to fetch lyrics from Musixmatch API
def get_lyrics(track_name, artist_name):
    
    api_key = ""
    api_url = "https://api.musixmatch.com/ws/1.1/matcher.lyrics.get"
    params = {
        "apikey": api_key,
        "q_track": track_name,
        "q_artist": artist_name,
        "f_has_lyrics": 1
    }

    try:
        response = requests.get(api_url, params=params)
        if response.status_code == 200:
            data = response.json()
            if data.get('message') and data['message'].get('body') and data['message']['body'].get('lyrics'):
                return data['message']['body']['lyrics']['lyrics_body']
            else:
                print(f"No lyrics found for '{track_name}' by '{artist_name}'")
                return None
        else:
            print(f"Error while fetching lyrics for '{track_name}' by '{artist_name}': {response.status_code} - {response.text}")
            return None

    except Exception as e:
        print(f"Exception while fetching lyrics for '{track_name}' by '{artist_name}': {str(e)}")
        return None

# Function to get audio features from Spotify
def get_audio_features(track_id, access_token):
    api_url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    try:
        response = requests.get(api_url, headers=headers)

        if response.status_code == 200:
            audio_features = response.json()
            return audio_features
        else:
            print(f"Error while fetching audio features for track ID '{track_id}': {response.status_code} - {response.text}")
            return None

    except Exception as e:
        print(f"Exception while fetching audio features for track ID '{track_id}': {str(e)}")
        return None

# Function to get audio analysis from Spotify
def get_audio_analysis(track_id, access_token):
    api_url = f"https://api.spotify.com/v1/audio-analysis/{track_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    try:
        response = requests.get(api_url, headers=headers)

        if response.status_code == 200:
            audio_analysis = response.json()
            return audio_analysis
        else:
            print(f"Error while fetching audio analysis for track ID '{track_id}': {response.status_code} - {response.text}")
            return None

    except Exception as e:
        print(f"Exception while fetching audio analysis for track ID '{track_id}': {str(e)}")
        return None

# Function to fetch recommendations for a list of seed tracks
def get_recommendations(seed_tracks, access_token):
    api_url = "https://api.spotify.com/v1/recommendations"
    params = {
        "seed_tracks": ",".join(seed_tracks),
        "limit": 10  
    }
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    try:
        response = requests.get(api_url, params=params, headers=headers)

        if response.status_code == 200:
            recommendations = response.json()
            return recommendations
        else:
            print(f"Error while fetching recommendations: {response.status_code} - {response.text}")
            return None

    except Exception as e:
        print(f"Exception while fetching recommendations: {str(e)}")
        return None


dataset = pd.read_csv('reduced_isrc.csv')


client_id = ""
client_secret = ""

# Initialize Spotify API client
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


track_info_list = []


seed_tracks_for_recommendations = []

# Iterate over the first 50 rows of the dataset
for index, row in dataset.head(50).iterrows():
    query = f"{row['name']} {row['artists']}"
    access_token = sp.auth_manager.get_access_token(as_dict=False)

   
    track_info = search_track(query, access_token)

    if track_info:
        track_id = track_info['id']
        seed_tracks_for_recommendations.append(track_id)
        
        
        lyrics = get_lyrics(row['name'], row['artists'])
        if lyrics:
            track_info['Lyrics'] = lyrics
        else:
            track_info['Lyrics'] = 'No lyrics found'

        # Fetch and store audio features
        audio_features = get_audio_features(track_id, access_token)
        if audio_features:
            # Separate audio feature components into different columns
            for feature_name, feature_value in audio_features.items():
                track_info[f'Audio Feature: {feature_name}'] = feature_value

        # Uncomment the following block to include Audio Analysis data
        # ==========================================================
        # Fetch and store audio analysis
        # audio_analysis = get_audio_analysis(track_id, access_token)
        # if audio_analysis:
        #     track_info['Audio Analysis'] = audio_analysis
        # ==========================================================

        track_info_list.append(track_info.copy())  


track_info_df = pd.DataFrame(track_info_list)


recommendations_list = []

# Fetch recommendations based on the seed tracks
for seed_track_id in seed_tracks_for_recommendations:
    try:
        recommendations = get_recommendations([seed_track_id], access_token)
        if recommendations and recommendations['tracks']:
            recommendations_list.append([rec['id'] for rec in recommendations['tracks']])
        else:
            recommendations_list.append(['Recommendation not found'])
    except Exception as e:
        print(f"Error while fetching recommendations: {str(e)}")
        recommendations_list.append(['Recommendation not found'])


track_info_df['Recommendations'] = recommendations_list


csv_file_name = 'spotify_track_info.csv'


track_info_df.to_csv(csv_file_name, index=False)

print("CSV file saved with recommendations.")

